# 开始

> tensorflow的学习成本有点高。而且。现在好像更多的人pytorch在做训练。改成pytorch吧。改天回来复习这一块，试一试能不能用上。
> 我发现我这种菜鸡，无论是什么框架都搞不出来。这几天，先把这个搞定。然后按着计划复习。

In [2]:
# 配置异步IO
import nest_asyncio
nest_asyncio.apply()

import collections

import numpy as np
import tensorflow as tf
import tensorflow_federated as tff

np.random.seed(0)

tff.federated_computation(lambda: 'Hello, World!')()

ModuleNotFoundError: No module named 'nest_asyncio'

In [ ]:
# 1 加载数据
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()

In [ ]:
# 1 查看数据内容

print(len(emnist_train.client_ids))
print(emnist_train.element_type_structure)

example_dataset = emnist_train.create_tf_dataset_for_client(
    emnist_train.client_ids[0])

example_element = next(iter(example_dataset))

print(example_element['label'].numpy())

In [5]:
from matplotlib import pyplot as plt
plt.imshow(example_element['pixels'].numpy(), cmap='gray', aspect='equal')
plt.grid(False)
_ = plt.show()

ImportError: DLL load failed while importing ft2font: 找不到指定的模块。

In [5]:
import matplotlib.pyplot as plt

ImportError: DLL load failed while importing ft2font: 找不到指定的模块。